# Image Vectorization using Traditional method

# step 0: import packages

In [1]:
# install packages
!pip install opencv-python
!pip install numpy
!pip install scipy
!pip install svgwrite

In [2]:
# import packages
import cv2
import numpy as np
from scipy.interpolate import splprep, splev
import svgwrite
import os

# step 1: Interpolation
use bilinear interpolation

In [18]:
# Read the image in grayscale format
image = cv2.imread('src/meditation.png', cv2.IMREAD_GRAYSCALE) # modify the input image
height, width = image.shape

scaleFactor = 2 # you can adjust this value and the interpolation method
resized_image = cv2.resize(image, (width * scaleFactor, height * scaleFactor), interpolation=cv2.INTER_CUBIC) # recommanded interpolation methods: cv2.INTER_NEAREST, cv2.INTER_LINEAR, cv2.INTER_CUBIC, cv2.INTER_LANCZOS4

# Save or display the resized image
#cv2.imwrite('output/resized_image.png', resized_image)

# find the threshold automatically based on the hostogram using Otsu's method
ret, thresholded_image = cv2.threshold(resized_image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# decide whether or not to invert the image
#inverted_image = 255 - thresholded_image
inverted_image = thresholded_image

# Find contours in the binary image
contours, _ = cv2.findContours(inverted_image, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)

# Create an empty image to draw contours on (make sure it's a 3-channel image)
contour_image = np.zeros((image.shape[0], image.shape[1], 3), dtype=np.uint8)

# Draw the contours on the empty image
# Using a visible color (e.g., green) and a suitable thickness
#cv2.drawContours(contour_image, contours, -1, (0, 255, 0), 1)  # Green color, thickness of 1

# Save or display the contour image
cv2.imwrite('output/intermediate_image.png', thresholded_image)
#cv2.imwrite('output/contour_image.png', contour_image)



True

# step 2: use contour points to generate SVG file

In [19]:
# Create SVG File
output_dir = 'output/meditation' # modify the output directory
os.makedirs(output_dir, exist_ok=True)
output_file = os.path.join(output_dir, 'meditation_cubic2.svg')
dwg = svgwrite.Drawing(output_file, profile='tiny')

for contour in contours:

    # Step 2: Fit Bezier Curves
    # Convert contour points to a format suitable for splprep
    points = contour[:, 0, :]  # Remove unnecessary dimension
    # make the number of contour points into 3n by deleting the last point
    if len(points) % 3 == 1:
        points = points[:-1]
    elif len(points) % 3 == 2:
        points = points[:-2]

    x, y = points[:, 0], points[:, 1]

    # Fit spline to contour points
    tck, u = splprep([x, y], s=0, k=3)

    # Evaluate spline and resample to ensure 3n points
    num_points = len(points)

    # Evaluate spline
    u_new = np.linspace(u.min(), u.max(), num=num_points)
    x_new, y_new = splev(u_new, tck)

    

    # Move to the start point
    path_data = f'M {x_new[0]},{y_new[0]} '

    # Create path data for cubic Bezier curve
    for i in range(1, len(x_new) - 1, 3):
        if i + 2 < len(x_new):
            path_data += f'C {x_new[i]},{y_new[i]} {x_new[i+1]},{y_new[i+1]} {x_new[i+2]},{y_new[i+2]} '
    # add the last curve
    path_data += f'C {x_new[num_points - 2]},{y_new[num_points - 2]} {x_new[num_points - 1]},{y_new[num_points - 1]} {x_new[0]},{y_new[0]} '

    # Add the path to the SVG
    dwg.add(dwg.path(d=path_data, stroke='black', fill='none'))

# Save the SVG file
dwg.save()
